In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 8.5 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

In [27]:
df_data_all = pd.read_csv("./KPIs.csv")

In [62]:
# Elimino las últimas tres columnas
df_data = df_data_all.iloc[ : , :-3]

In [64]:
arr_columns = df_data.iloc[0:0,:]
arr_columns

,CATEGORIA,TEMA,DATO BUSCADO,¿Se tiene?,ORIGEN,Campo agregado,Nombre del campo


## FILTRACIÓN POR LA COLUMNA CATEGORIA

In [168]:
categories = df_data['CATEGORIA'].unique()
categories

array(['Cap 1000', 'Energía eléctrica', 'Suministro de Agua',
       'Servicios Profesionales', 'Viáticos', 'Orden social', 'Inmuebles',
       'Papeleria', 'Alimentos', 'TI'], dtype=object)

In [202]:
category = categories[3]

In [203]:
df_data[df_data['CATEGORIA'] == category]

,CATEGORIA,TEMA,DATO BUSCADO,¿Se tiene?,ORIGEN,Campo agregado,Nombre del campo
25,Servicios Profesionales,Contratos,El total de trabajadores por área,No,Proveedores,Sí,Puesto de trabajo (Proveedores)
26,Servicios Profesionales,Contratos,El monto del contrato por área,Si,Facturas,NaN,NaN
27,Servicios Profesionales,Contratos,El total de contratos en hombres.,No,Proveedores,No,NaN
28,Servicios Profesionales,Contratos,El porcentaje de los contratos para hombres,No,NaN,No,NaN
29,Servicios Profesionales,Contratos,El total de contratos en mujeres,No,NaN,No,NaN
30,Servicios Profesionales,Contratos,El porcentaje de los contratos para mujeres,No,NaN,No,NaN
31,Servicios Profesionales,Contratos,El total de contratos.,No,NaN,No,NaN
32,Servicios Profesionales,Contratos,El costo total de todos los trabajadores,No,Facturas,No,NaN
33,Servicios Profesionales,Contratos,El costo total de todos los trabajadores del a...,No,Facturas,No,NaN
34,Servicios Profesionales,Contratos,La diferencia del costo total de los trabajado...,No,Facturas,No,NaN


## TEMA

In [204]:
arr_tema = df_data[df_data['CATEGORIA'] == category]['TEMA'].unique()

In [205]:
# Obtenemos los temas totales que tiene la categoria
arr_tema.size

1

In [206]:
# Se imprime el/los tema(s) de la categoria
for i in range(arr_tema.size):
    print(arr_tema[i])

Contratos


## Obtener Datos de cada tema

In [207]:
tema = arr_tema[0]

## Datos

In [208]:
data_total = df_data['¿Se tiene?'].loc[(df_data['CATEGORIA'] == category) & (df_data['TEMA'] == tema)].size
data_total

11

## Datos sin información

In [213]:
data_without_information = df_data['Campo agregado'].loc[(df_data['CATEGORIA'] == category) & (df_data['TEMA'] == tema) & (df_data['¿Se tiene?'] == 'No')].count()
data_without_information

10

## Datos con información

In [214]:
data_information = data_total - data_without_information
data_information

1